# Course Project - Convex Optimization for Signal Processing and Communications

- Student: Lucas von Ancken Garcia
- Matrikelnummer: 2576600 

## Problem 1

Given a single-user (point-to-point) MIMO communication channel with the channel matrix $H$ and zero mean
colored noise $n$ at the receiver antennas, the received symbol at the receiver is $y = H^H \cdot x + n$, where $x$ is the
transmitted symbol with zero mean. We want to **design the optimal transmit covariance matrix** that **maximizes the transmission rate** of the MIMO system. We assume that some of the **antennas have power constraints** that
restrict the transmitted power. Furthermore, **a sum power constraint applies**.

Generate required data to implement the problem formulation in CVX. Average the simulation results with Monte Carlo iterations.

In [35]:
import numpy as np
import cvxpy as cvx
from scipy import signal


np.random.seed(1)
np.set_printoptions(precision=4, suppress=True)

In [36]:
# Generation of the complex Channel Matrix H according to Rayleigh Fading Model
def generate_rayleigh_channel_matrix(m, n):
    real_part = np.random.normal(0, np.sqrt(0.5), (m,n))
    imag_part = np.random.normal(0, np.sqrt(0.5), (m,n))

    return real_part + 1j*imag_part

def generate_white_noise(n):
    real_part = np.random.normal(0, np.sqrt(0.5), n)
    imag_part = np.random.normal(0, np.sqrt(0.5), n)

    return real_part + 1j*imag_part

def generate_pink_noise_covariance(n):
        freqs = np.fft.fftfreq(n)
        psd = 1 / (np.abs(freqs) + 1e-4)
        psd = np.abs(psd)
        covariance_freq = np.fft.ifft(psd).real
        covariance_matrix = scipy.linalg.toeplitz(covariance_freq[:n])
        return covariance_matrix

In [37]:
def design_cov_matrix(H, Rn, power_limit_per_tx, total_power):
    num_tx, num_rx  = H.shape
    Rn_inv = np.linalg.inv(Rn)

    P = cvx.Variable((num_tx, num_tx))

    objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_inv @ H.conjugate().T @ P @ H))

    constraints = [P >> 0, cvx.trace(P) <= total_power]

    for i in range(num_tx):
        constraints.append(P[i, i] <= power_limit_per_tx[i])

    # Solve the problem
    prob = cvx.Problem(objective, constraints)
    prob.solve()

    return (prob.value, P.value)

In [38]:
num_tx = 5  # Number of transmit antennas
num_rx = 10  # Number of receive antennas

power_noise = 2
total_power = 50

power_limit_per_tx = [15, 20, 10, 8, 15]

H = cvx.Parameter((num_tx, num_rx), complex=True)
Rn_inv = cvx.Parameter((num_rx, num_rx))

P = cvx.Variable((num_tx, num_tx))

objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_inv @ cvx.conj(H.T) @ P @ H))

constraints = [P >> 0, cvx.trace(P) <= total_power]

for i in range(num_tx):
    constraints.append(P[i, i] <= power_limit_per_tx[i])

prob = cvx.Problem(objective, constraints)

H.value = generate_rayleigh_channel_matrix(num_tx, num_rx)
Rn_inv.value = np.identity(num_rx)*(1/power_noise)

prob.solve() 

17.39608368042657

In [39]:
print("Status:", prob.status)
print("Optimal value: ", prob.value)
print("Num of Constraints: ", len(constraints))
print("--------------------")
print("P* = ")
print(P.value)
print("Actual Total Power of the Optimal Solution: {}".format(P.value.trace()))
print("--------------------")
print("Lagrange Multipliers")
print("For P >= 0: ")
print(constraints[0].dual_value)
print("For total power:")
print(constraints[1].dual_value)
print("For the limit power of each TX:")
print([x.dual_value for x in constraints[2:]])

Status: optimal
Optimal value:  17.39608368042657
Num of Constraints:  7
--------------------
P* = 
[[10.6985  0.1051 -0.0548 -0.0485  0.0029]
 [ 0.1051 10.7931  0.0078 -0.0068 -0.0646]
 [-0.0548  0.0078 10.      0.0017 -0.1167]
 [-0.0485 -0.0068  0.0017  8.      0.0107]
 [ 0.0029 -0.0646 -0.1167  0.0107 10.5084]]
Actual Total Power of the Optimal Solution: 50.00000726671128
--------------------
Lagrange Multipliers
For P >= 0: 
[[-0.  0. -0.  0. -0.]
 [ 0. -0.  0.  0.  0.]
 [-0.  0. -0.  0.  0.]
 [ 0.  0.  0. -0. -0.]
 [-0.  0.  0. -0.  0.]]
For total power:
0.09081060183117554
For the limit power of each TX:
[0.0, 0.0, 0.005190696648534889, 0.026272380189099075, 0.0]


In [40]:
P_list = [P.value]
optimal_value_list = [prob.value]
for i in range(19):
    H.value = generate_rayleigh_channel_matrix(num_tx, num_rx)
    # optimal_value, P_optimal = design_cov_matrix(H, Rn, power_limit_per_tx, total_power)
    prob.solve()
    P_list.append(P.value)
    optimal_value_list.append(prob.value)

P_avg = sum(P_list)/len(P_list)
optimal_value_avg = sum(optimal_value_list)/len(optimal_value_list)

In [41]:
print("Optimal Value Average: {}".format(optimal_value_avg))
print("Average P* = ")
print(P_avg)

Optimal Value Average: 18.247619197017308
Average P* = 
[[10.6709  0.0013 -0.0123  0.0359 -0.0003]
 [ 0.0013 10.6585 -0.0327  0.0153 -0.012 ]
 [-0.0123 -0.0328 10.     -0.0817  0.0151]
 [ 0.0359  0.0153 -0.0817  8.     -0.0131]
 [-0.0003 -0.012   0.0151 -0.0131 10.6706]]


We assume further that a single-antenna co-channel user with channel vector hc is now present. The received symbol is $y_c = h^H_c \cdot x + n_c$, where $n_c is$ the zero mean colored noise at this user. We want that this user does not receive any interference in expectation, i.e., $E[∥y_c∥^2] = 0$.

Please modify the problem formulation to adopt the new constraint and solve it in CVX.

In [42]:
hc = generate_rayleigh_channel_matrix(num_tx, 1)
power_noise_c = 3

H = generate_rayleigh_channel_matrix(num_tx, num_rx)
Rn = np.identity(num_rx)*power_noise
Rn_inv = np.linalg.inv(Rn)

A = hc @ hc.conjugate().transpose()

P_new = cvx.Variable((num_tx, num_tx))

objective = cvx.Maximize(cvx.log_det(np.identity(num_rx) + Rn_inv @ H.conjugate().T @ P_new @ H))

constraints = [P_new >> 0, cvx.trace(P_new) <= total_power, cvx.trace(A @ P_new) == 0]

for i in range(num_tx):
    constraints.append(P_new[i, i] <= power_limit_per_tx[i])

prob = cvx.Problem(objective, constraints)
prob.solve(verbose=True)

print("Status:", prob.status)
print("Optimal value: ", prob.value)
print("Num of Constraints: ", len(constraints))

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 08 05:01:20 PM: Your problem has 25 variables, 32 constraints, and 0 parameters.
(CVXPY) Jun 08 05:01:20 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 08 05:01:20 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 08 05:01:20 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 08 05:01:20 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 08 05:01:20 PM: Compiling problem (target solver=SCS).
(CVX